# Lib

In [125]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
import glob

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")

from utils import utils_ml, parameters

from ipywidgets import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [126]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

18/11/2022  19:16    <DIR>          .
18/11/2022  16:18    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
18/11/2022  16:18         1.047.962 1.EA.ipynb
18/11/2022  20:25            67.359 2.Feature_engineering.ipynb
19/11/2022  02:54            22.885 3.Stability_var.ipynb
18/11/2022  19:08           131.212 4.Modeling.ipynb
19/11/2022  02:31    <DIR>          catboost_info
               4 arquivo(s)      1.269.418 bytes
               4 pasta(s)   341.844.910.080 bytes dispon�veis


# Reading

In [127]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [356]:
df = pd.read_parquet(".\\..\data\dados_transformed.parquet")
df['data'] = pd.to_datetime(df['data'])

In [357]:
features_bins_woe = utils_ml.list_subset_words( df.columns.tolist(), ["bin"])
## Excluding variable 'score' (score of Meli model) and 'o_obj' (with 72.5% of missing values))
features_bins_woe = utils_ml.exclude_words(features_bins_woe, ["score","o_obj"])
features_bins_woe

['n_boolean_bins',
 'p_boolean_bins',
 'categoria_produto_bins',
 'pais_bins',
 'a_int_bins',
 'b_float_bins',
 'c_float_bins',
 'd_float_bins',
 'e_float_bins',
 'f_float_bins',
 'h_float_bins',
 'k_float_bins',
 'l_float_bins',
 'm_float_bins',
 'monto_bins']

# Model

## Logistic regression

In [358]:
df_2_model = df[features_bins_woe + ["fraude"]]

### Dumming features

In [380]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

cat_vars = features_bins_woe

df_2_model["pais_hot_encode"] = le.fit_transform(df_2_model['pais_bins'])


# X = data.iloc[:, 0:4]
# y = data.iloc[:, 5]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

# classifier = LogisticRegression()
# classifier.fit(X_train, y_train)

C:\Users\caanp\AppData\Local\Temp\ipykernel_7748\4136534836.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_model["pais_hot_encode"] = le.fit_transform(df_2_model['pais_bins'])


In [384]:
df_2_model[["pais_hot_encode",'pais_bins']].value_counts()

pais_hot_encode  pais_bins
2                 1           111830
1                 0            37976
0                -1              194
dtype: int64

In [359]:
cat_vars = features_bins_woe

def transform( df , features = features_bins_woe, label = "fraude", drop_first = True):

    ls_df_dummies = []

    for var in cat_vars:

        ## creating dummy to variable
        df_dummy = pd.get_dummies( df[var] , prefix=var, drop_first = drop_first)
        ls_df_dummies.append(df_dummy)


    df_2_model_dum = pd.concat(ls_df_dummies + [ df[label] ], axis=1)

    return df_2_model_dum

In [360]:
df_2_model_dum = transform( df_2_model , features = features_bins_woe, label = "fraude")

In [361]:
cat_vars

['n_boolean_bins',
 'p_boolean_bins',
 'categoria_produto_bins',
 'pais_bins',
 'a_int_bins',
 'b_float_bins',
 'c_float_bins',
 'd_float_bins',
 'e_float_bins',
 'f_float_bins',
 'h_float_bins',
 'k_float_bins',
 'l_float_bins',
 'm_float_bins',
 'monto_bins']

### Train and test samples

In [362]:
from sklearn.model_selection import train_test_split

X = df_2_model_dum.drop(['fraude'], axis=1)
y = df_2_model_dum['fraude']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=0, stratify=y )

In [363]:
y.value_counts(normalize=True)[0]/y.value_counts(normalize=True)[1]

18.999999999999996

In [364]:
# X_train.columns.tolist()

### Trainning

In [365]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# lr = LogisticRegression(random_state=0,solver='newton-cg', fit_intercept=True, penalty='l2' )

# #Setting the range for class weights
# weights = np.linspace(0.0,0.99,200)

# #Creating a dictionary grid for grid search
# param_grid = {'class_weight': [{0:x, 1:1.0-x} for x in weights]}

# #Fitting grid search to the train data with 5 folds
# gridsearch = GridSearchCV(estimator= lr, 
#                           param_grid= param_grid,
#                           cv=StratifiedKFold(), 
#                           n_jobs=-1, 
#                           scoring='recall', 
#                           verbose=2).fit(X_train, y_train)

## Ploting the score for different values of weight
# sns.set_style('whitegrid')
# plt.figure(figsize=(12,8))
# weigh_data = pd.DataFrame({ 'score': gridsearch.cv_results_['mean_test_score'], 'weight': (1-weights)})
# sns.lineplot(weigh_data,  x='weight', y='score')
# plt.xlabel('Weight for class 1')
# plt.ylabel('F1 score')
# plt.xticks([round(i/10,1) for i in range(0,11,1)])
# plt.title('Scoring for different class weights', fontsize=24)

In [366]:
# LogisticRegression??

In [367]:
clf = LogisticRegression( random_state=0, solver='saga',l1_ratio = 0.1, fit_intercept=True, penalty='elasticnet', class_weight = "balanced" )

model = clf.fit(X_train, y_train )

c:\Users\caanp\miniconda3\envs\venv_ml\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [368]:
df_2_model['probabilty'] = model.predict_proba(X)[:,1]

C:\Users\caanp\AppData\Local\Temp\ipykernel_7748\2283094811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_model['probabilty'] = model.predict_proba(X)[:,1]


In [369]:
colunas_orig = utils_ml.exclude_words( df.columns.tolist(), ["bin","cat","WoE","fecha",'data','hora'] ) + ['categoria_produto']
colunas_orig

['a_int',
 'b_float',
 'c_float',
 'd_float',
 'e_float',
 'f_float',
 'pais',
 'h_float',
 'produto',
 'k_float',
 'l_float',
 'm_float',
 'n_boolean',
 'o_obj',
 'p_boolean',
 'monto',
 'score',
 'fraude',
 'categoria_produto']

In [370]:
df_ivs_ls = []

for variable in colunas_orig:

    label = "fraude"
    dtype_ = 'categorical' if df[variable].dtype=='O' or df[variable].dtype=='category' else 'numerical'

    df_iv = utils_ml.WOE_IV(df = df , variable = variable, dtype_ = dtype_ , label = label, monotonic= True, optimal_binning=True)[[ 'variable' , 'IV_var', 'KS']].drop_duplicates()

    df_ivs_ls.append(df_iv)

pd.concat(df_ivs_ls).sort_values("IV_var", ascending=False)

,variable,IV_var,KS
0,produto,6.369097,88.306667
0,categoria_produto,1.641776,39.766316
0,o_obj,1.094238,44.994386
0,score,1.021894,43.152982
0,f_float,0.447805,27.785965
0,l_float,0.439939,25.305965
0,n_boolean,0.361450,22.844211
0,m_float,0.337380,22.929825
0,p_boolean,0.245520,24.357193
0,d_float,0.170619,16.508070


In [371]:
coef_df = pd.DataFrame( {"Cols": model.feature_names_in_,  "coef": model.coef_[0] })
coef_df.sort_values("coef", ascending=False)

,Cols,coef
12,categoria_produto_bins_11,9.244559
11,categoria_produto_bins_10,8.144138
9,categoria_produto_bins_8,7.137946
6,categoria_produto_bins_5,7.079935
8,categoria_produto_bins_7,6.616204
4,categoria_produto_bins_3,6.452355
10,categoria_produto_bins_9,6.431652
3,categoria_produto_bins_2,5.819296
7,categoria_produto_bins_6,5.585527
5,categoria_produto_bins_4,5.259313


### comparing with Meli model

> **modelo MELI**

In [372]:
probability = "score"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,15000,"[0, 8]",417,14583,7500,142500,0,0,5.560000,10.233684,5.560000,10.233684,42.385965,1.000000,0.050000,0.000000,0.095238,0.208333
1,15000,"[8, 18]",692,14308,7083,127917,14583,417,9.226667,10.040702,14.786667,20.274386,42.385965,0.944400,0.052467,0.102337,0.099411,0.214636
2,15000,"[18, 28]",494,14506,6391,113609,28891,1109,6.586667,10.179649,21.373333,30.454035,42.385965,0.852133,0.053258,0.202744,0.100251,0.213033
3,15000,"[28, 38]",46,14954,5897,99103,43397,1603,0.613333,10.494035,21.986667,40.948070,42.385965,0.786267,0.056162,0.304540,0.104836,0.218407
4,15000,"[38, 48]",88,14912,5851,84149,58351,1649,1.173333,10.464561,23.160000,51.412632,42.385965,0.780133,0.065011,0.409481,0.120021,0.243792
5,15000,"[48, 58]",196,14804,5763,69237,73263,1737,2.613333,10.388772,25.773333,61.801404,42.385965,0.768400,0.076840,0.514126,0.139709,0.274429
6,15000,"[58, 68]",297,14703,5567,54433,88067,1933,3.960000,10.317895,29.733333,72.119298,42.385965,0.742267,0.092783,0.618014,0.164948,0.309278
7,15000,"[68, 78]",780,14220,5270,39730,102770,2230,10.400000,9.978947,40.133333,82.098246,42.385965,0.702667,0.117111,0.721193,0.200762,0.351333
8,15000,"[78, 88]",1283,13717,4490,25510,116990,3010,17.106667,9.625965,57.240000,91.724211,42.385965,0.598667,0.149667,0.820982,0.239467,0.374167


> **Logistic Regression**

In [373]:
probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_2_model, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,15000,"[2.9126573134859115e-09, 3.871302838709259e-06]",11,14989,7500,142500,0,0,0.146667,10.518596,0.146667,10.518596,42.596491,1.000000,0.050000,0.000000,0.095238,0.208333
1,15000,"[3.8713372841590675e-06, 2.748929256261448e-05]",30,14970,7489,127511,14989,11,0.400000,10.505263,0.546667,21.023860,42.596491,0.998533,0.055474,0.105186,0.105109,0.226939
2,15000,"[2.7502037512629473e-05, 0.00016114034348916945]",142,14858,7459,112541,29959,41,1.893333,10.426667,2.440000,31.450526,42.596491,0.994533,0.062158,0.210239,0.117004,0.248633
3,15000,"[0.00016115506188099809, 0.0005648255155468991]",307,14693,7317,97683,44817,183,4.093333,10.310877,6.533333,41.761404,42.596491,0.975600,0.069686,0.314505,0.130080,0.271000
4,15000,"[0.0005648467344649776, 0.0014841301479962487]",409,14591,7010,82990,59510,490,5.453333,10.239298,11.986667,52.000702,42.596491,0.934667,0.077889,0.417614,0.143795,0.292083
5,15000,"[0.0014841410896357554, 0.003629498786558885]",566,14434,6601,68399,74101,899,7.546667,10.129123,19.533333,62.129825,42.596491,0.880133,0.088013,0.520007,0.160024,0.314333
6,15000,"[0.0036297251039720878, 0.008944024717108782]",753,14247,6035,53965,88535,1465,10.040000,9.997895,29.573333,72.127719,42.596491,0.804667,0.100583,0.621298,0.178815,0.335278
7,15000,"[0.008944335300422988, 0.02439638565701219]",974,14026,5282,39718,102782,2218,12.986667,9.842807,42.560000,81.970526,42.596491,0.704267,0.117378,0.721277,0.201219,0.352133
8,15000,"[0.024396894785453205, 0.09405353661106185]",1437,13563,4308,25692,116808,3192,19.160000,9.517895,61.720000,91.488421,42.596491,0.574400,0.143600,0.819705,0.229760,0.359000


>> **Logistic Regression in test sample**

In [374]:
df_test = pd.DataFrame()
df_test['fraude'] = y_test
df_test['probabilty'] = model.predict_proba(X_test)[:,1]

probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_test, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,4500,"[1.797243763575067e-08, 4.053844008642206e-06]",5,4495,2250,42750,0,0,0.222222,10.514620,0.222222,10.514620,42.619883,1.000000,0.050000,0.000000,0.095238,0.208333
1,4500,"[4.056097465353607e-06, 2.9876429043392844e-05]",11,4489,2245,38255,4495,5,0.488889,10.500585,0.711111,21.015205,42.619883,0.997778,0.055432,0.105146,0.105029,0.226768
2,4500,"[2.9904590235681474e-05, 0.00016649005636767207]",45,4455,2234,33766,8984,16,2.000000,10.421053,2.711111,31.436257,42.619883,0.992889,0.062056,0.210152,0.116810,0.248222
3,4500,"[0.00016649967072257798, 0.0005770815140639721]",110,4390,2189,29311,13439,61,4.888889,10.269006,7.600000,41.705263,42.619883,0.972889,0.069492,0.314363,0.129719,0.270247
4,4500,"[0.0005771325497764546, 0.0015099143614398245]",115,4385,2079,24921,17829,171,5.111111,10.257310,12.711111,51.962573,42.619883,0.924000,0.077000,0.417053,0.142154,0.288750
5,4500,"[0.001509941102013066, 0.0036639616306127346]",156,4344,1964,20536,22214,286,6.933333,10.161404,19.644444,62.123977,42.619883,0.872889,0.087289,0.519626,0.158707,0.311746
6,4500,"[0.0036646393213051343, 0.009077990374308724]",222,4278,1808,16192,26558,442,9.866667,10.007018,29.511111,72.130994,42.619883,0.803556,0.100444,0.621240,0.178568,0.334815
7,4500,"[0.009078136431714278, 0.024143903918649446]",299,4201,1586,11914,30836,664,13.288889,9.826901,42.800000,81.957895,42.619883,0.704889,0.117481,0.721310,0.201397,0.352444
8,4500,"[0.02415134620058471, 0.09508205482213623]",429,4071,1287,7713,35037,963,19.066667,9.522807,61.866667,91.480702,42.619883,0.572000,0.143000,0.819579,0.228800,0.357500


## Catboost

In [20]:
# !pip install catboost

In [238]:
import catboost as cb

### Train and test samples

In [239]:
cat_features = [x for x in utils_ml.list_subset_words( df.columns.tolist(), ['cat']) if "WoE" not in x and "bin" not in x and x!='categoria_produto']

df_2_model_cat = df[cat_features + ["fraude"]]


X = df_2_model_cat.drop(['fraude'], axis=1)
y = df_2_model_cat['fraude']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=0, stratify=y )

### Trainning

In [240]:
model_cat_def = cb.CatBoostClassifier(
                           l2_leaf_reg = 3.0,
                           scale_pos_weight = 19,
                           verbose=False,
                           thread_count = 8,
                           random_seed = 0
                          )


modelo_cat = model_cat_def.fit(X_test , y_test, cat_features =  cat_features  )

In [241]:
coef = pd.DataFrame( {"features":modelo_cat.feature_names_,"coef":modelo_cat.feature_importances_})
coef = coef.sort_values("coef", ascending=False).reset_index(drop=True).query("coef>0")
coef

,features,coef
0,categoria_produto_cat,35.722931
1,o_obj_cat,10.118526
2,l_float_cat,7.436246
3,monto_cat,7.341126
4,score_cat,6.984067
5,f_float_cat,6.945596
6,b_float_cat,4.165394
7,d_float_cat,4.118669
8,m_float_cat,3.693161
9,h_float_cat,3.123742


### comparing with Meli model

> **modelo MELI**

In [242]:
probability = "score"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,15000,"[0, 8]",417,14583,7500,142500,0,0,5.560000,10.233684,5.560000,10.233684,42.385965,1.000000,0.050000,0.000000,0.095238,0.208333
1,15000,"[8, 18]",692,14308,7083,127917,14583,417,9.226667,10.040702,14.786667,20.274386,42.385965,0.944400,0.052467,0.102337,0.099411,0.214636
2,15000,"[18, 28]",494,14506,6391,113609,28891,1109,6.586667,10.179649,21.373333,30.454035,42.385965,0.852133,0.053258,0.202744,0.100251,0.213033
3,15000,"[28, 38]",46,14954,5897,99103,43397,1603,0.613333,10.494035,21.986667,40.948070,42.385965,0.786267,0.056162,0.304540,0.104836,0.218407
4,15000,"[38, 48]",88,14912,5851,84149,58351,1649,1.173333,10.464561,23.160000,51.412632,42.385965,0.780133,0.065011,0.409481,0.120021,0.243792
5,15000,"[48, 58]",196,14804,5763,69237,73263,1737,2.613333,10.388772,25.773333,61.801404,42.385965,0.768400,0.076840,0.514126,0.139709,0.274429
6,15000,"[58, 68]",297,14703,5567,54433,88067,1933,3.960000,10.317895,29.733333,72.119298,42.385965,0.742267,0.092783,0.618014,0.164948,0.309278
7,15000,"[68, 78]",780,14220,5270,39730,102770,2230,10.400000,9.978947,40.133333,82.098246,42.385965,0.702667,0.117111,0.721193,0.200762,0.351333
8,15000,"[78, 88]",1283,13717,4490,25510,116990,3010,17.106667,9.625965,57.240000,91.724211,42.385965,0.598667,0.149667,0.820982,0.239467,0.374167


> **Catboost model**

In [243]:
df_2_model_cat['probabilty'] = modelo_cat.predict_proba(X)[:,1]

probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_2_model_cat, probability, label, quantil)

C:\Users\caanp\AppData\Local\Temp\ipykernel_7748\240659597.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_model_cat['probabilty'] = modelo_cat.predict_proba(X)[:,1]


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,15000,"[7.71438019804436e-05, 0.003229909900620665]",2,14998,7500,142500,0,0,0.026667,10.524912,0.026667,10.524912,66.161404,1.000000,0.050000,0.000000,0.095238,0.208333
1,15000,"[0.003230323923035018, 0.019109729315543125]",2,14998,7498,127502,14998,2,0.026667,10.524912,0.053333,21.049825,66.161404,0.999733,0.055541,0.105249,0.105235,0.227212
2,15000,"[0.01911159540564689, 0.06807829472070138]",44,14956,7496,112504,29996,4,0.586667,10.495439,0.640000,31.545263,66.161404,0.999467,0.062467,0.210498,0.117584,0.249867
3,15000,"[0.06807848878797317, 0.10462216311880024]",71,14929,7452,97548,44952,48,0.946667,10.476491,1.586667,42.021754,66.161404,0.993600,0.070971,0.315453,0.132480,0.276000
4,15000,"[0.10462396059257056, 0.14279666111778228]",94,14906,7381,82619,59881,119,1.253333,10.460351,2.840000,52.482105,66.161404,0.984133,0.082011,0.420218,0.151405,0.307542
5,15000,"[0.1427996880596146, 0.19426974989348558]",160,14840,7287,67713,74787,213,2.133333,10.414035,4.973333,62.896140,66.161404,0.971600,0.097160,0.524821,0.176655,0.347000
6,15000,"[0.19427341732585965, 0.2878895694010516]",301,14699,7127,52873,89627,373,4.013333,10.315088,8.986667,73.211228,66.161404,0.950267,0.118783,0.628961,0.211170,0.395944
7,15000,"[0.28789241708578867, 0.44679991824370036]",612,14388,6826,38174,104326,674,8.160000,10.096842,17.146667,83.308070,66.161404,0.910133,0.151689,0.732112,0.260038,0.455067
8,15000,"[0.44681204040695166, 0.6582698615124198]",1174,13826,6214,23786,118714,1286,15.653333,9.702456,32.800000,93.010526,66.161404,0.828533,0.207133,0.833081,0.331413,0.517833


>> **Catboost in test sample**

In [244]:
df_test = pd.DataFrame()
df_test['fraude'] = y_test
df_test['probabilty'] = modelo_cat.predict_proba(X_test)[:,1]

probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_test, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,4500,"[7.71438019804436e-05, 0.0033114219263442315]",0,4500,2250,42750,0,0,0.000000,10.526316,0.000000,10.526316,73.263158,1.000000,0.050000,0.000000,0.095238,0.208333
1,4500,"[0.003311967130632306, 0.02132902520975196]",0,4500,2250,38250,4500,0,0.000000,10.526316,0.000000,21.052632,73.263158,1.000000,0.055556,0.105263,0.105263,0.227273
2,4500,"[0.02136735142666924, 0.0691711294313374]",0,4500,2250,33750,9000,0,0.000000,10.526316,0.000000,31.578947,73.263158,1.000000,0.062500,0.210526,0.117647,0.250000
3,4500,"[0.0691719215307923, 0.10587070575688665]",4,4496,2250,29250,13500,0,0.177778,10.516959,0.177778,42.095906,73.263158,1.000000,0.071429,0.315789,0.133333,0.277778
4,4500,"[0.10588341680994681, 0.14317445610432633]",2,4498,2246,24754,17996,4,0.088889,10.521637,0.266667,52.617544,73.263158,0.998222,0.083185,0.420959,0.153573,0.311944
5,4500,"[0.14317451087846314, 0.19469737698094183]",21,4479,2244,20256,22494,6,0.933333,10.477193,1.200000,63.094737,73.263158,0.997333,0.099733,0.526175,0.181333,0.356190
6,4500,"[0.19474452909832513, 0.29135547696908115]",66,4434,2223,15777,26973,27,2.933333,10.371930,4.133333,73.466667,73.263158,0.988000,0.123500,0.630947,0.219556,0.411667
7,4500,"[0.29138207776819824, 0.4481734687705463]",141,4359,2157,11343,31407,93,6.266667,10.196491,10.400000,83.663158,73.263158,0.958667,0.159778,0.734667,0.273905,0.479333
8,4500,"[0.4482869866001156, 0.6582260022999138]",346,4154,2016,6984,35766,234,15.377778,9.716959,25.777778,93.380117,73.263158,0.896000,0.224000,0.836632,0.358400,0.560000


## Using period in evaluation

> **modelo MELI**

Apresenta uma relativa estabilidade quando se olha os score para as semanas

In [245]:
gb = df.groupby(pd.Grouper(key='data', freq="W"))

dff_with_week = []

for week, df_week in gb:

    df_week['week'] = week

    dff_with_week.append(df_week)


dff_week = pd.concat(dff_with_week)

C:\Users\caanp\AppData\Local\Temp\ipykernel_7748\26454910.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_week['week'] = week
C:\Users\caanp\AppData\Local\Temp\ipykernel_7748\26454910.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_week['week'] = week
C:\Users\caanp\AppData\Local\Temp\ipykernel_7748\26454910.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [246]:
probability = "score"
label="fraude"
quantil=10

@interact

def meli_model(week = weeks):

    df_marco = dff_week[dff_week['week']==week]

    display(utils_ml.metric_evaluation(df_marco, probability, label, quantil))

interactive(children=(Dropdown(description='week', options=(numpy.datetime64('2020-03-08T00:00:00.000000000'),…

> **Catboost**

Apresenta uma relativa estabilidade no tempo quando se olha para as semanas

In [247]:
probability = "probabilty"
label="fraude"
quantil=10

@interact

def cat_model(week = weeks):

    df_marco_cat = dff_week[dff_week['week']==week][cat_features+['fraude']]

    df_marco_cat['probabilty'] = modelo_cat.predict_proba( df_marco_cat )[:,1]

    display(utils_ml.metric_evaluation(df_marco_cat, probability, label, quantil))

interactive(children=(Dropdown(description='week', options=(numpy.datetime64('2020-03-08T00:00:00.000000000'),…

> **Logistic Regression**

Apresenta instabilidade quando se escore por semana e se olha para os decis

In [377]:
def transform_2_model(df , features = features_bins_woe, label ="fraude", cols_model = df_2_model_dum.columns.tolist() ):

    ## tirando a coluna label caso esteja na lista
    cols_model = utils_ml.exclude_words(cols_model, [label] )
    ## Transformando para dummies
    df_marco_model_dum = transform(df = df, features = features_bins_woe, label ="fraude")

    ## Preenchendo colunas faltantes
    dummies_faltantes = list(   set(cols_model) - set(df_marco_model_dum.columns.to_list()) )

    if len(dummies_faltantes)>0:

        for dummi_falt in dummies_faltantes:

            df_marco_model_dum[dummi_falt] = 0

    return df_marco_model_dum[cols_model+[label]]

In [378]:
probability = "probabilty"
label="fraude"
quantil=10

@interact

def cat_model(week = weeks):

    df = dff_week[dff_week['week']==week]

    df_marco_model_dum = transform_2_model(df , features = features_bins_woe, label ="fraude", cols_model = df_2_model_dum.columns.tolist())

    ######### Fitting ###########################
    df_marco_model_dum['probabilty'] = model.predict_proba( df_marco_model_dum.drop("fraude", axis=1) )[:,1]


    display(utils_ml.metric_evaluation(df_marco_model_dum, probability, label, quantil))


interactive(children=(Dropdown(description='week', options=(numpy.datetime64('2020-03-08T00:00:00.000000000'),…

In [38]:
!lsb_release -a

'lsb_release' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


# Requirements

In [110]:
path_root = parameters.root
!pip list --format=freeze > $path_root/requirements.txt

5697.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
